In [1]:
%pip install icecream

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from icecream import ic
print=ic

# for von-misses stresses
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib import colormaps


## ELEMENTS, SYSTEM, SOLVER and PLOTTER

In [4]:
class HeatElement2D:
    def  __init__(self, node_indeces, node_coordinates):
        self.node_indeces = node_indeces
        self.node_coordinates = node_coordinates

        lamb = 1
        mu = 0.5

        # # plain strain assumption
        # self.C = np.array([
        #     [lamb+2*mu, lamb, 0],
        #     [lamb, lamb+2*mu, 0],
        #     [0,0,mu]])

        # plain stress assumption
        self.C = np.array([
            [4*mu*(mu+lamb), 2*mu*lamb, 0],
            [2*mu*lamb, 4*mu*(mu+lamb), 0],
            [0,0,lamb*mu + 2*mu**2]
        ]) / (lamb + 2*mu)



        self.K_elem_global = self.get_local_stiffness_matrix() # local isn't actually local, it is in parametric square space in range [-1, 1] for both x and y

    def get_global_stiffness_matrix(self):
        return self.get_local_stiffness_matrix()

    def get_local_stiffness_matrix(self):
        self.k_elem_local = np.zeros([len(self.node_indeces)*2, len(self.node_indeces)*2]) # 8x8
        self.gauss_quadrature_points = [
            [-1/np.sqrt(3), -1/np.sqrt(3)],
            [+1/np.sqrt(3), -1/np.sqrt(3)],
            [+1/np.sqrt(3), +1/np.sqrt(3)],
            [-1/np.sqrt(3), +1/np.sqrt(3)],
            ]
        self.gauss_quadrature_weights = [1,1,1,1]

        for i, (ksi1_g, ksi2_g) in enumerate(self.gauss_quadrature_points):
            B = self.get_B(ksi1_g, ksi2_g)
            wk = self.gauss_quadrature_weights[i]
            j = np.linalg.det(self.get_jacobian(ksi1_g, ksi2_g))
            self.k_elem_local += B.T @ self.C @ B * j * wk

        return self.k_elem_local

    def get_B(self, ksi1_g, ksi2_g):
        J = self.get_jacobian(ksi1_g, ksi2_g)
        J_inv = np.linalg.inv(J)
        J_inv[J_inv == 0] = 1e-12

        # B_{parameter_index}_{shape_function}
        B11 = dN1_ksi1(ksi1_g, ksi2_g) * J_inv[0][0] + dN1_ksi2(ksi1_g, ksi2_g) * J_inv[1][0]
        B12 = dN2_ksi1(ksi1_g, ksi2_g) * J_inv[0][0] + dN2_ksi2(ksi1_g, ksi2_g) * J_inv[1][0]
        B13 = dN3_ksi1(ksi1_g, ksi2_g) * J_inv[0][0] + dN3_ksi2(ksi1_g, ksi2_g) * J_inv[1][0]
        B14 = dN4_ksi1(ksi1_g, ksi2_g) * J_inv[0][0] + dN4_ksi2(ksi1_g, ksi2_g) * J_inv[1][0]

        B21 = dN1_ksi1(ksi1_g, ksi2_g) * J_inv[0][1] + dN1_ksi2(ksi1_g, ksi2_g) * J_inv[1][1]
        B22 = dN2_ksi1(ksi1_g, ksi2_g) * J_inv[0][1] + dN2_ksi2(ksi1_g, ksi2_g) * J_inv[1][1]
        B23 = dN3_ksi1(ksi1_g, ksi2_g) * J_inv[0][1] + dN3_ksi2(ksi1_g, ksi2_g) * J_inv[1][1]
        B24 = dN4_ksi1(ksi1_g, ksi2_g) * J_inv[0][1] + dN4_ksi2(ksi1_g, ksi2_g) * J_inv[1][1]

        B = np.array([
            [B11, 0, B12, 0, B13, 0, B14, 0],
            [0, B21, 0, B22, 0, B23, 0, B24],
            [B21, B11, B22, B12, B23, B13, B24, B14]
        ])

        return B


    def get_jacobian(self, ksi1, ksi2):
        XY = self.node_coordinates

        X = XY[:,0]; Y = XY[:,1]

        J11 = dN1_ksi1(ksi1, ksi2) * X[0] + dN2_ksi1(ksi1, ksi2) * X[1] + dN3_ksi1(ksi1, ksi2) * X[2] + dN4_ksi1(ksi1, ksi2) * X[3]
        J12 = dN1_ksi2(ksi1, ksi2) * X[0] + dN2_ksi2(ksi1, ksi2) * X[1] + dN3_ksi2(ksi1, ksi2) * X[2] + dN4_ksi2(ksi1, ksi2) * X[3]
        J21 = dN1_ksi1(ksi1, ksi2) * Y[0] + dN2_ksi1(ksi1, ksi2) * Y[1] + dN3_ksi1(ksi1, ksi2) * Y[2] + dN4_ksi1(ksi1, ksi2) * Y[3]
        J22 = dN1_ksi2(ksi1, ksi2) * Y[0] + dN2_ksi2(ksi1, ksi2) * Y[1] + dN3_ksi2(ksi1, ksi2) * Y[2] + dN4_ksi2(ksi1, ksi2) * Y[3]

        self.J = np.array([[J11, J12], [J21, J22]])
        return self.J

## utils
# shape functions
def N1(ksi1, ksi2): return 1/4 * (1-ksi1)*(1-ksi2)
def N2(ksi1, ksi2): return 1/4 * (1+ksi1)*(1-ksi2)
def N3(ksi1, ksi2): return 1/4 * (1+ksi1)*(1+ksi2)
def N4(ksi1, ksi2): return 1/4 * (1-ksi1)*(1+ksi2)

# derivatives of shape functions
def dN1_ksi1(ksi1, ksi2): return -1/4 * (1-ksi2)
def dN1_ksi2(ksi1, ksi2): return -1/4 * (1-ksi1)
def dN2_ksi1(ksi1, ksi2): return 1/4 * (1-ksi2)
def dN2_ksi2(ksi1, ksi2): return -1/4 * (1+ksi1)
def dN3_ksi1(ksi1, ksi2): return 1/4 * (1+ksi2)
def dN3_ksi2(ksi1, ksi2): return 1/4 * (1+ksi1)
def dN4_ksi1(ksi1, ksi2): return -1/4 * (1+ksi2)
def dN4_ksi2(ksi1, ksi2): return 1/4 * (1-ksi1)


In [5]:
class HeatSystem2D:
    def __init__(self, elements, num_nodes, forces, boundary_conditions):
        self.elements = elements
        self.num_nodes = num_nodes
        self.forces = forces
        self.boundary_conditions = boundary_conditions

In [6]:
class Solver:
    def __init__(self, system):
        self.system = system

    def solve(self):
        K_global, F_global = self.assemble_global_matrices()
        Kuu, Fu = self.apply_boundary_conditions()
        displacements = self.solve_displacements()
        reaction_forces = self.get_reaction_forces()
        strains, stresses = self.calculate_loads()

        # return self.displacements, self.loads

    def assemble_global_matrices(self):
        # Assemble global stiffness
        self.K_global = np.zeros((self.system.num_nodes*2, self.system.num_nodes*2))

        for element in self.system.elements:
            K_elem = element.get_global_stiffness_matrix()
            node_per_element = len(self.system.elements[0].node_indeces) # 2 for linear

            # Get the global DOF indices for the element's nodes
            global_dof_indices = []
            for node_index in element.node_indeces:
                global_dof_indices.extend([2 * node_index, 2 * node_index + 1])  # DOFs: x, y

            for i in range(len(global_dof_indices)):
                for j in range(len(global_dof_indices)):
                    self.K_global[global_dof_indices[i], global_dof_indices[j]] += K_elem[i, j]

        # Assemble load vector
        self.F_global = np.zeros((self.system.num_nodes*2))
        for node_index, f_x, f_y in self.system.forces:
            global_dof_index = int(2 * node_index)
            self.F_global[global_dof_index] += f_x
            self.F_global[global_dof_index + 1] += f_y

        return self.K_global, self.F_global

    def apply_boundary_conditions(self):
        known_global_indeces = []
        for node_index, constraint_x, constraint_y in self.system.boundary_conditions:
            global_dof_index = int(2 * node_index)
            if constraint_x == 1:
                known_global_indeces.append(global_dof_index)
            if constraint_y == 1:
                known_global_indeces.append(global_dof_index + 1)

        self.known_global_indeces = known_global_indeces
        self.unknown_global_indeces = [i for i in range(self.system.num_nodes*2) if i not in self.known_global_indeces]

        # get the matrix while remowing the indeces in known global indeces
        self.Kuu = np.delete(np.delete(self.K_global, known_global_indeces, axis=0), known_global_indeces, axis=1)
        self.Fu = np.delete(self.F_global, known_global_indeces, axis=0)

        return self.Kuu, self.Fu

    def solve_displacements(self):
        self.Du = np.linalg.solve(self.Kuu, self.Fu) # unknown displacements

        # merge known and unknown displacements
        self.displacements = np.zeros((self.system.num_nodes*2))
        self.displacements[self.unknown_global_indeces] = self.Du

        return self.displacements

    def get_reaction_forces(self):
         # The reaction forces can be solved from the equation F_k = K_{ku} D_u
        K_ku = self.K_global[self.known_global_indeces, :] # Rows corresponding to known DOFs
        K_ku = K_ku[:, np.setdiff1d(np.arange(self.K_global.shape[1]), self.known_global_indeces)] # Columns corresponding to unknown DOFs
        self.Fk = K_ku @ self.Du
        return self.Fk

    def calculate_loads(self):
        self.stresses = np.zeros((len(self.system.elements), 3)) # sigma11, sigma22, sigma12
        self.strains = np.zeros((len(self.system.elements), 3)) # epsilon11, epsilon22, epsilon12

        for i, element in enumerate(self.system.elements):

            pos1, pos2, pos3, pos4 = element.node_coordinates

            node_index1, node_index2, node_index3, node_index4 = element.node_indeces

            dx1, dy1 = self.displacements[2 * node_index1:2 * node_index1 + 2]
            dx2, dy2 = self.displacements[2 * node_index2:2 * node_index2 + 2]
            dx3, dy3 = self.displacements[2 * node_index3:2 * node_index3 + 2]
            dx4, dy4 = self.displacements[2 * node_index4:2 * node_index4 + 2]

            d_element = np.array([dx1, dy1, dx2, dy2, dx3, dy3, dx4, dy4])

            B = np.zeros((3, 8))
            for j, (ksi1_g, ksi2_g) in enumerate(element.gauss_quadrature_points):
                B += element.get_B(ksi1_g, ksi2_g)

            strain_element = B @ d_element

            self.strains[i] = strain_element
            self.stresses[i] = element.C @ strain_element
            self.stresses[i][2] /= 2

        return self.strains, self.stresses


In [7]:
class Plotter:
    def __init__(self, solver):
        self.solver = solver
        self.system = solver.system
        self.displacements = solver.displacements

        # self.stresses = solver.stresses

    def plot_system(self):
        """
        Plots the truss system with stress visualization inside displaced elements.
        """
        fig, ax = plt.subplots(figsize=(8, 8))

        # Create a colormap for stress visualization
        stress_values = self.solver.stresses.flatten()  # Flatten to consider all stress values
        norm = mcolors.Normalize(vmin=stress_values.min(), vmax=stress_values.max())
        cmap = colormaps['viridis']
        # cmap = colormaps['inferno']

        for i, element in enumerate(self.system.elements):
            pos1, pos2, pos3, pos4 = element.node_coordinates
            node_index1, node_index2, node_index3, node_index4 = element.node_indeces
            offset = 0  # Offset for text

            # Extract initial coordinates for all 4 nodes
            x1, y1 = pos1
            x2, y2 = pos2
            x3, y3 = pos3
            x4, y4 = pos4

            # PLOT SYSTEM WITH DISPLACEMENTS
            dx1, dy1 = self.displacements[2 * node_index1:2 * node_index1 + 2]
            dx2, dy2 = self.displacements[2 * node_index2:2 * node_index2 + 2]
            dx3, dy3 = self.displacements[2 * node_index3:2 * node_index3 + 2]
            dx4, dy4 = self.displacements[2 * node_index4:2 * node_index4 + 2]

            x1_disp = x1 + dx1
            y1_disp = y1 + dy1
            x2_disp = x2 + dx2
            y2_disp = y2 + dy2
            x3_disp = x3 + dx3
            y3_disp = y3 + dy3
            x4_disp = x4 + dx4
            y4_disp = y4 + dy4

            # Plot the displaced element outline
            ax.plot([x1_disp, x2_disp, x3_disp, x4_disp, x1_disp], [y1_disp, y2_disp, y3_disp, y4_disp, y1_disp],
                    c="b", linestyle="--")

            # Extract stresses for this element
            sigma_x, sigma_y, sigma_xy = self.solver.stresses[i]

            # Combine stresses for visualization (e.g., von Mises stress)
            von_mises_stress = ((sigma_x ** 2 - sigma_x * sigma_y + sigma_y ** 2 + 3 * sigma_xy ** 2) ** 0.5)

            # Compute the color for this element based on stress
            color = cmap(norm(von_mises_stress))

            # Create a filled polygon for stress visualization
            polygon = patches.Polygon(
                [[x1_disp, y1_disp], [x2_disp, y2_disp], [x3_disp, y3_disp], [x4_disp, y4_disp]],
                closed=True, facecolor=color, edgecolor="black"
            )
            ax.add_patch(polygon)

        # Add colorbar to show stress mapping
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label('Stress (von Mises)')

        plt.title("Frame Element(s) with Stress Visualization")
        plt.xlabel("x [mm]")
        plt.ylabel("y [mm]")
        plt.axis('equal')
        plt.show()

## SOLUTION

In [15]:
nodal_coordinates = np.array([
    [0. , 0. ],
    [0.5, 0. ],
    [1. , 0. ],
    [0. , 0.5],
    [0.5, 0.5],
    [1. , 0.5],
    [0. , 1. ],
    [0.5, 1. ],
    [1. , 1. ]])


connectivity=np.array([
    [0,1,2,3],
    [4,5,6,7]
])

# node_index, constraint_T
boundary_conditions = np.array([[5,1], [6,1], [7,1]])

T = 1/6
 # node_index, T
forces = np.array([ # temperatures
    [5, T],
    [6, T],
    [7, T]
    ])

num_nodes = len(nodal_coordinates)

elements = []
for i, node_indeces in enumerate(connectivity):
    node_coordinates = nodal_coordinates[node_indeces]
    element = HeatElement2D(node_indeces, node_coordinates)
    elements.append(element)

system = HeatSystem2D(elements, num_nodes, forces, boundary_conditions)

In [17]:
solver = Solver(system)
solver.assemble_global_matrices()

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
solver = Solver(system)
solver.solve()
# _ = print(solver.K_global, solver.F_global, solver.Kuu, solver.Fu, solver.Du, solver.displacements)

# for i in range(0, len(solver.displacements), 2):
#     nodal_disp = f"Node {i//2}: del_x = {solver.displacements[i]:.4f} mm, del_y = {solver.displacements[i+1]:.4f} mm"
#     print(nodal_disp)

In [18]:
# plotter = Plotter(solver)
# plotter.plot_system()